In [1]:
import pandas as pd
import random
import re
from IPython.display import clear_output
import itertools

In [2]:
#create an index file for the huge data file
def createIndex(filePath):
    file = open(filePath,'r');
    indexFilePath = filePath+"_index.csv"
    indexFile = open(indexFilePath,'w');
    indexFile.close()
    
    offset = 0
    lineNumber = 0
    for line in file:
#         clear_output(wait=True)
#         print(lineNumber)
        indexFile = open(indexFilePath,'a');
        indexFile.write(str(lineNumber) + "," + str(offset) + "\n")
        indexFile.close()
        lineNumber = lineNumber + 1
        offset += len(line)
    file.close()
    return indexFilePath

In [3]:
def generateNIndecesFrom(n, rangeOfIndeces):
    print("Generating " + str(n) + " indeces from range")
    allIndeces = random.sample(rangeOfIndeces, n)
    allIndeces = pd.Series(data = allIndeces)
    allIndeces = allIndeces.sort_values().reset_index().drop(['index'],axis=1)
    allIndeces.columns = ['Index'];
    return allIndeces

In [4]:
def generateAndSaveSubset(file,dataFrame,indexValues,numRowsPerItteration,totalNumRows,column_headers,frameSaveName):
    dataFrame = generateSubSet(file,dataFrame,indexValues,numRowsPerItteration,totalNumRows,column_headers)
    dataFrame.to_csv(frameSaveName)
    return dataFrame

In [16]:
def generateSubSet(file,dataFrame,indexValues,numRowsPerItteration,totalNumRows,column_headers):
    totalNumIterations = int(totalNumRows/numRowsPerItteration)
    print("Number of itterations = " + str(totalNumIterations))
    totalNumRowsTraversed = 0
    prevsize = 0
    for i in range(totalNumIterations + 1):
#         
#         print("Itteration number: " + str(i))
#         print("skipRows: " + str(i * numRowsPerItteration))
#         print("Read in : " + str(numRowsPerItteration))
        curData = pd.read_table(file,skiprows = i * numRowsPerItteration, nrows = numRowsPerItteration,header=None)
        curData.index = [i for i in range(i*numRowsPerItteration,i*numRowsPerItteration + curData.shape[0])]
        totalNumRowsTraversed = totalNumRowsTraversed + curData.shape[0]

#         print(curData.shape)
#         print(curData.index.shape)

        
        curData.columns = column_headers
        curData['Index'] = curData.index
#         print(indexValues)
#         break
        curIndexRange = indexValues['Index'][(indexValues['Index'] < (i*numRowsPerItteration + numRowsPerItteration)) & (indexValues['Index'] > (i*numRowsPerItteration-1))]
        curData = curData[curData['Index'].isin(curIndexRange)]

        dataFrame = pd.concat([dataFrame,curData])
        
        clear_output()
        print("Extraction Stats: " + str(dataFrame.shape[0]) + " percent: " + str(dataFrame.shape[0] / indexValues.shape[0] * 100) + "%")
        print("Document Stats: " + str(totalNumRowsTraversed) + " percent: " + str(totalNumRowsTraversed/totalNumRows*100) + "%")
        if (dataFrame.shape[0] - prevsize) > 500000:
            prevsize = dataFrame.shape[0]
#             dataFrame.to_csv(frameSaveName)
        elif dataFrame.shape[0] == indexValues.shape[0]:
            print("Finished with the data collection")
#             dataFrame.to_csv(frameSaveName)
            break
    print("Extraction is Done, now saving frame")        
    return dataFrame

In [11]:
#Load or Generate the 2M indeces to use
try:
    twoMIndeces = pd.read_csv('2MIndeces.csv',squeeze = True)
    
except:
    print("There were not 2000000 data points")
    twoMIndeces = generateNIndecesFrom(2000000,range(0,45840617)) # this range is because there are this number of records in the training set.
    twoMIndeces.to_csv('2MIndeces.csv',index=False,header=False)

There were not 2000000 data points
Generating 2000000 indeces from range


In [33]:
#Load or Generate the 1M indeces for train, 250k validation and 750k test
try:
    trainIndeces = pd.read_csv('train_ids.txt',squeeze = True)
    validationIndeces = pd.read_csv('validation_ids.txt',squeeze = True)
except:
    print("There were not 1000000 data points")
    trainIndeces = generateNIndecesFrom(1000000,list(twoMIndeces['Index']))
    trainIndeces.to_csv('train_ids.txt',index=False,header=False)

    remainingIndeces = twoMIndeces['Index'][~twoMIndeces['Index'].isin(trainIndeces.values)]
    validationIndeces = generateNIndecesFrom(250000,list(remainingIndeces))
    validationIndeces.to_csv('validation_ids.txt',index=False,header=False)
    
    testingIndeces = twoMIndeces['Index'][~(twoMIndeces['Index'].isin(trainIndeces.values) | twoMIndeces['Index'].isin(validationIndeces.values))]
    testingIndeces = generateNIndecesFrom(750000,list(testingIndeces))
    testingIndeces.to_csv('testing_ids.txt',index=False,header=False)


There were not 1000000 data points
Generating 1000000 indeces from range
Generating 250000 indeces from range
Generating 750000 indeces from range


In [13]:
def getColumnHeaders():
    return pd.Series(data=['Index','label','integer_1','integer_2','integer_3',
                                 'integer_4','integer_5','integer_6','integer_7','integer_8','integer_9'
                                 'integer_10','integer_11','integer_12','integer_13','categorical_1',
                                 'categorical_2','categorical_3','categorical_4','categorical_5','categorical_6',
                                 'categorical_7','categorical_8','categorical_9','categorical_10','categorical_11',
                                 'categorical_12','categorical_13','categorical_14','categorical_15','categorical_16',
                                 'categorical_17','categorical_18','categorical_19','categorical_20','categorical_21',
                                 'categorical_22','categorical_23','categorical_24','categorical_25','categorical_26'])
column_headers = getColumnHeaders()

In [40]:
try:
    train1M = pd.read_csv('train1M.csv',squeeze = True)
except:
    print("No 1M collection")
    train1M = pd.DataFrame()
    train1M = generateAndSaveSubset('dac/train.txt',train1M,trainIndeces,4000000,46000000,column_headers,'train1M.csv')

Extraction Stats: 1000000 percent: 100.0%
Document Stats: 45840617 percent: 99.6535152173913%
Finished with the data collection
Extraction is Done, now saving frame


In [41]:
try:
    validation250k = pd.read_csv('validation250k.csv',squeeze = True)
except:
    print("No 250k collection")
    validation250k = pd.DataFrame()
    validation250k = generateAndSaveSubset('dac/train.txt',validation250k,validationIndeces,4000000,46000000,column_headers,'validation250k.csv')

Extraction Stats: 250000 percent: 100.0%
Document Stats: 45840617 percent: 99.6535152173913%
Finished with the data collection
Extraction is Done, now saving frame


In [42]:
try:
    test750k = pd.read_csv('test750k.csv',squeeze = True)
except:
    print("No 750k collection")
    test750k = pd.DataFrame()
    test750k = generateAndSaveSubset('dac/train.txt',test750k,validationIndeces,4000000,46000000,column_headers,'test750k.csv')

Extraction Stats: 250000 percent: 100.0%
Document Stats: 45840617 percent: 99.6535152173913%
Finished with the data collection
Extraction is Done, now saving frame


In [43]:
train1M.shape

(1000000, 40)

In [46]:
train1M.head()

,Index,label,integer_1,integer_2,integer_3,integer_4,integer_5,integer_6,integer_7,integer_8,...,categorical_17,categorical_18,categorical_19,categorical_20,categorical_21,categorical_22,categorical_23,categorical_24,categorical_25,categorical_26
14,14,0.0,51,84.0,4.0,3633.0,26.0,1.0,4.0,8.0,...,3486227d,12195b22,21ddcdc9,b1252a9d,fa131867,NaN,dbb486d7,8ecc176a,e8b83407,c43c3f58
147,147,0.0,1,14.0,2.0,120.0,733.0,0.0,12.0,606.0,...,8efede7f,97029569,21ddcdc9,b1252a9d,6ec2bcf7,NaN,55dd3565,7f686ab3,2bf691b1,bebc2875
206,206,0.0,76,3.0,NaN,5029.0,NaN,NaN,16.0,NaN,...,d4bb7bd8,2804effd,NaN,NaN,723b4dfd,NaN,3a171ecb,b34f3128,NaN,NaN
211,211,NaN,-1,NaN,NaN,3123.0,22.0,48.0,0.0,32.0,...,27c07bd6,e5f8f18f,NaN,NaN,f3ddd519,NaN,32c7478e,b34f3128,NaN,NaN
241,241,NaN,25,1.0,NaN,17330.0,62.0,1.0,3.0,76.0,...,07c540c4,04d863d5,21ddcdc9,b1252a9d,a3a82059,78e2e389,423fab69,45ab94c8,e8b83407,c84c4aec


In [44]:
validation250k.shape

(250000, 40)

In [45]:
test750k.shape

(250000, 40)